## Lookbehind and Lookaheads

In many cases, you want to match for some specific text, but make sure that it is **either right before** another pattern, or **right after** another pattern. For instance, we can use it to match multi-line dialogue:

### Task: Get all of Yu Chen's Dialogue

In [2]:
dialogue = '''
YUCHEN: You can reach me at ychen220@marshall.usc.edu.
If you cannot find me there; you can try my other email.
You can also call my assistant Todd.
MIKE: Sounds good. Well then,
it's been a pleasure.
YUCHEN: Nice.
MIKE: Good talk.
JIMMY: Yeah, it's been a great experience all around.
YUCHEN: Okay, good bye.
'''

import re

My first attempt will involve what we have already learned. The `[\w ;@\.,]+` says we want to match any alphanumeric character (`\w`), and white space (` `), and semicolon, @, period, or comma: (`;@\.,` ) at least one or more times.

In [3]:
##capture group shows what you want to match after the keyword of the person you want the dialogue from
##match all the thing in brackets, want 1 or more
##THIS DOES NOT MATCH THE 2ND AND 3RD LINES OF THE FIRST DIALOGUE
re.findall(r'YUCHEN: ([A-Za-z0-9 ;@\.,]+)', dialogue)

['You can reach me at ychen220@marshall.usc.edu.', 'Nice.', 'Okay, good bye.']

Notice that right now, this regular expression capture almost all of my dialogue, but not everything. It misses the `If you cannot find me there, you can try my other email.` in the second line, since it doesn't understand it needs to match stuff after a new line. I can add in `\n` to my brackets so it knows to match beyond a new line:

In [4]:
## \w is letter number or underscore
##then have characters AND newlines
##but then you add MIKE, which you don't want
re.findall(r'YUCHEN: ([\w @;\n\.,]+)', dialogue)

['You can reach me at ychen220@marshall.usc.edu.\nIf you cannot find me there; you can try my other email.\nYou can also call my assistant Todd.\nMIKE',
 'Nice.\nMIKE',
 'Okay, good bye.\n']

Notice now that the problem is `MIKE`, the next character speaking, gets dragged into the match, along with his dialogue. This is because the regex does not know that `Mike: Sounds good.` is another person speaking - it thinks it is still part of `YUCHEN`'s dialogue.

In [5]:
##the capture group in the () with the \w IS SAME AS BEFORE
##? means a special type of group, ?: means non-capture group
##for this, the first one is a POSTIIVE LOOKBEHIND GROUP (you want this to appear here, it's BEHIND the text you
##care about, "this has to be here"), <= is bakcwards arrow, look behind
##after the capture group, it's a POSITIVE LOOKAHEAD GROUP, looking ahead of capture group, with ?=
##both are forms of a non-capture group

##return everything that matches the pattern, but it HAS to be between YUCHEN and MIKE
re.findall(r'(?<=YUCHEN: )([\w @;\n\.,]+)(?=\nMIKE: )', dialogue)

##STILL, when YUCHEN is last speaker it doesn't pick that up

['You can reach me at ychen220@marshall.usc.edu.\nIf you cannot find me there; you can try my other email.\nYou can also call my assistant Todd.',
 'Nice.']

The `(?<=YUCHEN: )` is called a **positive lookbehind**, and it states that whatever we match, it must begin with (but not include) `YUCHEN: `. The `(?=\nMIKE: )` is a **positive lookahead**, and it states that whatever we match, it must end with `\nMIKE: ` (a new line, then Mike beginning to speak).

This is looks great, but we are still missing the final line in the dialogue (`Okay, good bye.`). That currently no longer matches, since it does not have `\nMIKE: ` at its end. We can add a conditional to our positive lookahead to account for the fact that it is the end of the text:`(?=\nMIKE: |$)`- this states that we either need to have the end of the entire string, or we need to have `\nMIKE: ` after our match:

In [56]:
##in my pos lookahead, it either is newline followed by MIKE, OR $ (end of the string)

##if you did ! instead of =, it's a NEGATIVE lookahead (more complicated)
##basically makes it so that you can say NOT preceded by YUCHEN or NOT followed by MIKE if it's a negative
re.findall(r'(?<=YUCHEN: )([\w @;\n\.,]+)(?=\nMIKE: |$)', dialogue)

['You can reach me at ychen220@marshall.usc.edu.\nIf you cannot find me there; you can try my other email.\nYou can also call my assistant Todd.',
 'Nice.',
 'Okay, good bye.\n']

## Greedy versus Lazy Evaluation

By default, regular expressions try to match as MUCH as possible- **this is called greedy evaluation**. Sometimes, we only want to match for as little as possible- this is called **lazy evaluation**. We can do this by putting a `?` after our quantifier (remember a quantifier is `+`,`*`, or `{3,5}` - anything that tells regex how many times to match something). For instance:

In [59]:
sentence = "Hello"
import re

##. is any character, 0 or more, with an l, match H, 0 or more character till L, so Hell (GREEDY EVAL)
re.findall("H.*l", sentence) #matches Hell

##with ?, it will try to match as few characters as possible, as soon as it finds a match it exits (LAZY EVAL)
##matches H, match anything (e), 0 or more of those, then an L, doesn't add extra L
re.findall("H.*?l", sentence) #matches Hel

['Hel']

In [12]:
s = "Caller"
re.findall("C.*l", s) #matches Hell

['Call']

In [13]:

re.findall("C.*?l", s) #matches Hel

['Cal']

If we apply this lazy evaluation. to our original dialogue `(?<=YUCHEN: )([\w @;\n\.,]+?)(?=\nMIKE: |$)`, we notice that the last match no longer has the `\n` attached to it. This is because it matches only the minimum text sufficient, and the `\n` symbol is not necessary to complete the match:

In [6]:
re.findall(r'(?<=YUCHEN: )([\w @;\n\.,]+?)(?=\nMIKE: |$)', dialogue) # notice the ? before the plus sign

['You can reach me at ychen220@marshall.usc.edu.\nIf you cannot find me there; you can try my other email.\nYou can also call my assistant Todd.',
 'Nice.',
 'Okay, good bye.']

Another example: let's say we are trying to find all HTML tags in a particular website:

In [66]:
html = '''
<HTML>
        <HEAD>
            <TITLE>Your Title Here</TITLE>
        </HEAD>
    <a href="http://somegreatsite.com">Link Name</a>
    <H1>This is a Header</H1>
    <H2>This is a Medium Header</H2>
<</HTML>
'''

In [67]:
import re

##text between the tags gets dragged in
re.findall(r'<.+>', html)

['<HTML>',
 '<HEAD>',
 '<TITLE>Your Title Here</TITLE>',
 '</HEAD>',
 '<a href="http://somegreatsite.com">Link Name</a>',
 '<H1>This is a Header</H1>',
 '<H2>This is a Medium Header</H2>',
 '<</HTML>']

This drags in everything between the tags themselves, including the text `Your Title Here` and `This is a Header`. We don't want that. This is because the regex `<.+>` is trying to match as much of the text as possible. However, using lazy quantifiers, we can get regex to match only the minimum necessary:

In [68]:
##this way, you can JUST get the tags
re.findall(r'<.+?>', html)

['<HTML>',
 '<HEAD>',
 '<TITLE>',
 '</TITLE>',
 '</HEAD>',
 '<a href="http://somegreatsite.com">',
 '</a>',
 '<H1>',
 '</H1>',
 '<H2>',
 '</H2>',
 '<</HTML>']